In [ ]:
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as opt
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, auc, roc_curve
from copy import copy, deepcopy
import zipfile
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
'''torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True'''

Mounted at /content/drive


'torch.cuda.set_device(0)\ntorch.backends.cudnn.benchmark = True'

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/text classification/BBC News/bbc-text.csv')
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [ ]:
df['category'].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

'''example_text = ['I will watch Memento tonight']
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, truncation=True, return_tensors="pt")
tokenizer.decode(bert_input.input_ids[1])'''
text_column, out_column = 'text', 'category'
labels = dict(zip(df[out_column].unique(), range(df[out_column].nunique())))
df.replace({out_column: labels}, inplace=True)
df_train, df_valid, df_test = np.split(df.sample(frac=1, random_state=42),  [int(.8*len(df)), int(.9*len(df))])
df_train.shape, df_valid.shape, df_test.shape

((1780, 2), (222, 2), (223, 2))

In [ ]:
df_train

,category,text
414,4,brown and blair face new rift claims for the u...
420,1,small firms hit by rising costs rising fuel ...
1644,3,spirit awards hail sideways the comedy sideway...
416,0,microsoft releases patches microsoft has warne...
1232,2,arsenal through on penalties arsenal win 4-2 o...
...,...,...
801,2,ireland 19-13 england ireland consigned englan...
1774,0,warning over tsunami aid website net users are...
512,0,digital guru floats sub-$100 pc nicholas negro...
633,3,gallery unveils interactive tree a christmas t...


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class BertLoader(Dataset):
    def __init__(self, encoded_data, labels):
        self.labels = labels
        self.encoded_data = encoded_data

    def classes(self):
        return self.labels
 
    def __len__(self):
        return len(self.encoded_data['input_ids'])
 
    def __getitem__(self, idx):
        return self.encoded_data['input_ids'][idx], self.encoded_data['attention_mask'][idx], self.labels[idx] if self.labels is not None else None

class NNModel(nn.Module):
    def __init__(self, input_shape, units=None, factors=None, activ=True, norm=False, dropout=False, slops=None):
        super().__init__()
        self.input_shape = input_shape
        self.units = units
        self.factors = factors
        self.activ, self.norm = activ, norm
        self.network = nn.ModuleList()
        if self.factors:
            self.units = np.round(self.input_shape * np.asarray(self.factors)).astype(int)
        if self.units is not None:
            self.dropout = np.zeros_like(self.units) if not dropout else dropout
            self.slops = np.full(len(self.units), 1) if slops is None else slops
            for i, j, k in zip(self.units, self.dropout, self.slops):
                if i >= 1:
                    block = self.__build_block__(input_shape, i, p=j, slop=k)
                    self.network.extend(block)
                    input_shape = i
        self.output_shape = input_shape
        self.reset_parameters()
    
    def __build_block__(self, input_shape, units, p, slop):
        block = []
        block.append(nn.Linear(input_shape, units, bias=not self.norm))
        if self.norm:
            block.append(nn.BatchNorm1d(units))
            #block.append(nn.LayerNorm(units, eps=1e-5))
        if self.activ:
            #block.append(nn.LeakyReLU())
            block.append(nn.ELU(slop))
            #block.append(nn.GELU())
        if p > 0:
            block.append(nn.Dropout(p))
        return block
 
    def forward(self, x):
        for layer in self.network:
          tmp = layer(x)
          x = tmp
        return x
 
    def reset_parameters(self):
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                layer.bias.data.fill_(0.1)
 

class BertClassifier(nn.Module):
    def __init__(self, mlp_units, mlp_dropout, nb_class):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.mlp = NNModel(768, units=mlp_units, factors=None, dropout=[mlp_dropout]*len(mlp_units)) if mlp_units is not None else None
        cls_units = self.mlp.output_shape if mlp_units is not None else 768
        self.classifier = nn.Linear(cls_units, nb_class)

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        z = self.mlp(pooled_output) if self.mlp is not None else pooled_output
        pred = self.classifier(z)
        return z, pooled_output, pred

class BaseBertClassifier:
    def __init__(self, model, tokenizer):
        self.model = model.to(device)
        self.losses = {'Epoch': [], 'Train': [], 'Test': [], 'BState': [], 'LState': [], 'LR': []}
        self.tokenizer = tokenizer
 
    def train_model(self, optim, train_loader, grad_clip, l2_reg):
          total_loss = 0
          self.model = self.model.train()
        #with autograd.detect_anomaly():
          for i, (ids, mask, Y) in enumerate(train_loader):
              ids, mask, Y = ids.to(device), mask.to(device), Y.to(device)
              #self.model.get_weight()
              optim.zero_grad()
              loss = self.loss_function(ids, mask, Y, l2_reg)
              loss.backward()
              torch.nn.utils.clip_grad_norm_(self.model.parameters(), grad_clip)
              optim.step()
              total_loss += loss.item()
          return total_loss/(i+1)
        
 
    def eval_model(self, test_loader):
        self.model = self.model.eval()
        total_loss = 0
        for i, (ids, mask, Y) in enumerate(test_loader):
            ids, mask, Y = ids.to(device), mask.to(device), Y.to(device)
            loss = self.loss_function(ids, mask, Y, l2_reg=0)
            total_loss += loss.item()
        return total_loss/(i+1)#np.abs(-100. - total_loss)
 
    def fit(self, X_train, Y_train, epoch, lr, opt_kwarg, batch_size=None,  grad_clip=100, momentum=0.9, X_test=None, Y_test=None, l2_reg=0, verbose=True, save=True):
        batch_size = len(Y_train) if batch_size is None else batch_size
        encoded_train = self.tokenizer.batch_encode_plus(X_train, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=512, return_tensors='pt')
        train_load = DataLoader(BertLoader(encoded_train, Y_train), batch_size=batch_size, shuffle=True)  # DATALOADER obj
        if X_test is not None:
            encoded_test = self.tokenizer.batch_encode_plus(X_test, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=512, return_tensors='pt')
            test_load = DataLoader(BertLoader(encoded_test, Y_test), batch_size=batch_size, shuffle=True)
 
        best_loss = 1e100
        optim = opt.Adam(self.model.parameters(), lr=lr)
        #optim = opt.SGD(self.model.parameters(), lr=lr, momentum=momentum, nesterov=True)

        scheduler = None
        #scheduler = opt.lr_scheduler.CyclicLR(optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.ReduceLROnPlateau(optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.MultiStepLR(optim, milestones=[28, 120], gamma=0.1)

        eval_score = ''
        for i in range(epoch):
            if verbose:
                print('##### EPOCH ' + str(i) + ' #####')
               
            train_loss = self.train_model(optim, train_load, grad_clip, l2_reg)
            self.losses['LState'] = deepcopy(self.model.state_dict())
    
            if verbose:
                print('train loss : ', train_loss)
            self.losses['Epoch'].append(i), self.losses['Train'].append(train_loss)
    
            if df_test is not None:
                valid_loss = self.eval_model(test_load)

                if verbose:
                    print('test loss : ', valid_loss)
                self.losses['Test'].append(valid_loss)
    
                if scheduler is not None:
                    scheduler.step(valid_loss)
                    self.losses['LR'].append(optim.param_groups[0]['lr'])
                    '''scheduler.step()
                    self.losses['LR'].append(scheduler.get_last_lr()[0])'''
    
                if valid_loss < best_loss:
                    self.losses['BState'] = deepcopy(self.model.state_dict())
                    best_loss = valid_loss
                    print('===========SAVE===========')


class Multiclass(BaseBertClassifier):#multiclass classification
    def __init__(self, model, tokenizer):
        super(Multiclass, self).__init__(model, tokenizer,)

    def loss_function(self, input_id, mask, Y, l2_reg):
        _, _, pred = self.model(input_id, mask)
        ce_loss = nn.CrossEntropyLoss()
        loss = ce_loss(pred, Y)
        return loss

    def prdict(self, X, batch_size):
        self.model.eval()
        encoded_data = self.tokenizer.batch_encode_plus(X, add_special_tokens=True, return_attention_mask=True, pad_to_max_length=True, max_length=512, return_tensors='pt')
        data_load = DataLoader(TensorDataset(encoded_data['input_ids'],  encoded_data['attention_mask']),batch_size=batch_size)
        outputs = {'z': [], 'pooled_output': [], 'pred': []}
        for i, (ids, mask) in enumerate(data_load):
            ids, mask = ids.to(device), mask.to(device)
            z, pooled_output, pred = self.model(ids, mask)
            pred = nn.Softmax()(pred)
            z, pooled_output, pred = z.cpu().data.numpy(), pooled_output.cpu().data.numpy(), pred.cpu().data.numpy()
            outputs['z'].extend(z), outputs['pooled_output'].extend(pooled_output), outputs['pred'].extend(pred)
        return outputs

def gradient_clipper(model: nn.Module, val: float) -> nn.Module:
    def process_grad(grad):
        grad[grad != grad] = 1e-10
        return torch.clamp(grad, -val, val)
    for parameter in model.parameters():
        parameter.register_hook(lambda grad: process_grad(grad))
    
    return model

In [ ]:
epoch, lr, batch_size, d, mlp_d = 50000, 5e-5, 8, 0.000001, 1e-6
#cyclic_kwarg = {'base_lr': lr, 'max_lr': 1e-2, 'step_size_up':200, 'step_size_down':200}
plateau_kwarg = {'factor':0.5, 'patience':200, 'verbose':True, 'min_lr':1e-7, 'mode':'min'}

model = BertClassifier(mlp_units=None, mlp_dropout=1e-6, nb_class=len(labels))
model = gradient_clipper(model, 10)
#nn_model.load_state_dict(best_state)
print(device)
print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))
bert_mc = Multiclass(model, tokenizer)
bert_mc.fit(df_train[text_column].values, df_train[out_column].values, epoch, lr, plateau_kwarg, batch_size=batch_size, grad_clip=10, momentum=0.9,
        X_test=df_valid[text_column].values, Y_test=df_valid[out_column].values, l2_reg=0, verbose=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda
BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


##### EPOCH 0 #####
train loss :  0.33954120386441994
test loss :  0.12012455471059573
===========SAVE===========
##### EPOCH 1 #####
train loss :  0.11365090809227907
test loss :  0.22097229595029994
##### EPOCH 2 #####
train loss :  0.07116246003670108
test loss :  0.08009558678272047
===========SAVE===========
##### EPOCH 3 #####
train loss :  0.03621406607813256
test loss :  0.07369448415452748
===========SAVE===========
##### EPOCH 4 #####
train loss :  0.023090836575574286
test loss :  0.0917876655834594
##### EPOCH 5 #####
train loss :  0.010507119596871076
test loss :  0.10657569291236411
##### EPOCH 6 #####


KeyboardInterrupt: ignored

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

best_state = deepcopy(bert_mc.losses['BState'])
bert_mc.model.load_state_dict(best_state)
print(np.min(bert_mc.losses['Test']))

fig = make_subplots(rows=3, cols=1)
s = 0
fig.append_trace(go.Scatter(x=bert_mc.losses['Epoch'][s:], y=bert_mc.losses['Train'][s:],mode='lines',name='Train'), row=1, col=1)
fig.append_trace(go.Scatter(x=bert_mc.losses['Epoch'][s:], y=bert_mc.losses['Test'][s:],mode='lines',name='Test'), row=2, col=1)
fig.append_trace(go.Scatter(x=bert_mc.losses['Epoch'][s:], y=bert_mc.losses['LR'][s:],mode='lines',name='LR'), row=3, col=1)
fig.update_layout(height=1000, width=1500, title_text="Stacked Subplots")
fig.show()

0.07369448415452748


In [ ]:
output = bert_mc.prdict(df_valid[text_column].values, 2)
pred = np.argmax(np.asarray(output['pred']), 1)
acc = accuracy_score(df_valid[out_column].values, pred)
acc, confusion_matrix(df_valid[out_column].values, pred)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).

<ipython-input-8-92e956f6224f>:176: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



(0.9819819819819819,
 array([[34,  0,  0,  1,  0],
        [ 2, 38,  0,  0,  0],
        [ 0,  0, 56,  0,  0],
        [ 0,  0,  0, 42,  0],
        [ 0,  1,  0,  0, 48]]))

In [ ]:
output = bert_mc.prdict(df_test[text_column].values, 2)
pred = np.argmax(np.asarray(output['pred']), 1)
acc = accuracy_score(df_test[out_column].values, pred)
acc, confusion_matrix(df_test[out_column].values, pred)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).

<ipython-input-8-92e956f6224f>:176: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



(0.9820627802690582,
 array([[38,  1,  0,  1,  0],
        [ 1, 55,  0,  0,  1],
        [ 0,  0, 50,  0,  0],
        [ 0,  0,  0, 33,  0],
        [ 0,  0,  0,  0, 43]]))